# Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense

In [2]:
songs = pd.read_csv("song_list5.csv")

In [3]:
songs.head()

,songid,artist,track,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,5X4Qm0rVLcZeeO4tSDmBg3,Jack Bruce,Running Thro' Our Hands,0.456,0.255,9.0,-15.805,1.0,0.0480,0.94600,0.170000,0.951,0.0532,116.424,253067.0,4.0
1,1pNpt53PZPet9dvJN3RKGr,Prefuse 73,Parachute Panador,0.535,0.806,7.0,-10.289,1.0,0.0642,0.00436,0.019100,0.457,0.3760,90.089,63733.0,4.0
2,3oxz2oCzAWdPzA6In2zA5u,Pasion Vega,La Gata Bajo La Lluvia,0.294,0.482,5.0,-6.406,1.0,0.0430,0.46300,0.000000,0.335,0.2040,166.693,255280.0,3.0
3,05JGVUwt7XJk5FPqH0Wsch,Jonny Lang,Walking Away,0.563,0.631,0.0,-5.144,1.0,0.0324,0.06350,0.000008,0.163,0.5400,115.657,254827.0,4.0
4,3xdgCFMTn6ut8fZYxfAuR0,Skye,All the Promises,0.358,0.611,2.0,-9.752,0.0,0.0454,0.51500,0.000468,0.149,0.1640,171.596,256933.0,4.0


In [4]:
songs.dtypes

songid               object
artist               object
track                object
danceability        float64
energy              float64
key                 float64
loudness            float64
mode                float64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
duration_ms         float64
time_signature      float64
dtype: object

In [7]:
features = songs[[
    "danceability", "energy", "key", "loudness", "mode", "speechiness", 
    "acousticness", "instrumentalness", "liveness", "valence", "tempo", 
    "duration_ms", "time_signature"]].to_numpy()

In [8]:
features[0]

array([ 4.56000e-01,  2.55000e-01,  9.00000e+00, -1.58050e+01,
        1.00000e+00,  4.80000e-02,  9.46000e-01,  1.70000e-01,
        9.51000e-01,  5.32000e-02,  1.16424e+02,  2.53067e+05,
        4.00000e+00])

In [10]:
scaler = StandardScaler()

In [11]:
x_train = scaler.fit_transform(features)

In [13]:
print(x_train[0])
print(x_train[0].shape)

[-0.47695143 -1.43616823  1.03467011 -1.32564479  0.70711739 -0.34028656
  1.91669133 -0.14321669  3.87500921 -1.70714343 -0.17066299  0.03404049
  0.24368604]
(13,)


In [33]:
encoder_input = Input(shape=(13,))

encoder_dense1 = Dense(10, activation='relu')(encoder_input)
encoder_dense2 = Dense(8, activation='relu')(encoder_dense1)
encoder_dense3 = Dense(6, activation='relu')(encoder_dense2)
encoder_output = Dense(4, activation='relu')(encoder_dense3)

encoder = keras.Model(encoder_input, encoder_output)



decoder_dense1 = Dense(6, activation='relu')(encoder_output)
decoder_dense2 = Dense(8, activation='relu')(decoder_dense1)
decoder_dense3 = Dense(10, activation='relu')(decoder_dense2)
decoder_output = Dense(13, activation='sigmoid')(decoder_dense3)

auto_encoder = keras.Model(encoder_input, decoder_output)

auto_encoder.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 13)]              0         
_________________________________________________________________
dense_18 (Dense)             (None, 10)                140       
_________________________________________________________________
dense_19 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_20 (Dense)             (None, 6)                 54        
_________________________________________________________________
dense_21 (Dense)             (None, 4)                 28        
_________________________________________________________________
dense_22 (Dense)             (None, 6)                 30        
_________________________________________________________________
dense_23 (Dense)             (None, 8)                 56  

In [34]:
auto_encoder.compile(optimizer='nadam',
                    loss='mean_squared_error',
                    metrics=['accuracy'])

In [35]:
auto_encoder.fit(x_train, x_train,
                epochs=100,
                batch_size=32,
                shuffle=True)

Epoch 1/100
1563/1563 [==============================] - 2s 1ms/step - loss: 0.9406 - accuracy: 0.1116
Epoch 2/100
1563/1563 [==============================] - 2s 1ms/step - loss: 0.8377 - accuracy: 0.2696
Epoch 3/100
1563/1563 [==============================] - 2s 1ms/step - loss: 0.8110 - accuracy: 0.3038
Epoch 4/100
1563/1563 [==============================] - 2s 1ms/step - loss: 0.7952 - accuracy: 0.3394
Epoch 5/100
1563/1563 [==============================] - 2s 1ms/step - loss: 0.7843 - accuracy: 0.3913
Epoch 6/100
1563/1563 [==============================] - 2s 1ms/step - loss: 0.7747 - accuracy: 0.4106
Epoch 7/100
1563/1563 [==============================] - 2s 1ms/step - loss: 0.7692 - accuracy: 0.4255
Epoch 8/100
1563/1563 [==============================] - 2s 1ms/step - loss: 0.7657 - accuracy: 0.4333
Epoch 9/100
1563/1563 [==============================] - 2s 1ms/step - loss: 0.7637 - accuracy: 0.4346
Epoch 10/100
1563/1563 [==============================] - 2s 1ms/step - l

In [42]:
auto_encoder.save('auto_encoder')

INFO:tensorflow:Assets written to: auto_encoder\assets


In [38]:
# auto_encoder.save("/nnmodels/auto_encoder")

filepath = "/nnmodels/auto_encoder"

tf.keras.models.save_model(
    auto_encoder, filepath, overwrite=True, include_optimizer=True, save_format=None,
    signatures=None, options=None
)

INFO:tensorflow:Assets written to: /nnmodels/auto_encoder\assets


In [ ]:
# from tensorflow import keras
# model = keras.models.load_model('path/to/location')